# ***GNN Model***
1. Get Data from spreadsheet.
2. Train model (150 epochs).
3. Export model to Gdrive.
4. Deploy to R4B.
5. Display to Blynk IoT app.

In [ ]:
#install libraries
!pip install torch_scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cpu.html
!pip install torch_sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cpu.html
!pip install torch_cluster -f https://pytorch-geometric.com/whl/torch-1.9.0+cpu.html
!pip install torch_spline_conv -f https://pytorch-geometric.com/whl/torch-1.9.0+cpu.html
!pip install torch_geometric
!pip install torch torch_geometric pandas gspread oauth2client

**Complete Code**

In [ ]:
#import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
!pip install torch_geometric
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
import gspread
from oauth2client.service_account import ServiceAccountCredentials
#mount the google drive
from google.colab import drive
drive.mount('/content/gdrive')
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Step 1: Read data from Google Spreadsheet
spreadsheet_name = 'FYP (RoboLab)'  # Replace with your spreadsheet name
worksheet_name = 'Sheet1'  # Replace with your worksheet name

# Open the specified spreadsheet and worksheet
spreadsheet = gc.open(spreadsheet_name)
worksheet = spreadsheet.worksheet(worksheet_name)

# Extract the data from the worksheet
data = worksheet.get_all_values()
headers = data[0]
data = data[1:]

# Convert data to a DataFrame
df = pd.DataFrame(data, columns=headers)
data = df.iloc[:, 2:].values.astype(float)  # Extract columns 3 to 11 (sensor readings)
# edge_index = torch.tensor([[0, 1, 2, 3],
#                            [1, 0, 3, 2],
#                            [4, 5, 6],
#                            [5, 4, 6],
#                            [7, 8, 9],
#                            [8, 7, 9]])
edge_index = torch.tensor([
    [0, 2], [0, 3], [0, 4], [0, 5],  # Sensor fusion 1 (0) connected to sensors 2, 3, 4, 5
    [1, 2], [1, 3], [1, 4], [1, 5],  # Sensor fusion 2 (1) connected to sensors 2, 3, 4, 5
    [6, 8], [6, 9], [6, 10],         # Sensor fusion 3 (6) connected to sensors 8, 9, 10
    [7, 8], [7, 9], [7, 10],         # Sensor fusion 4 (7) connected to sensors 8, 9, 10
])

edge_index = edge_index.t().contiguous()  # Transpose and make contiguous



# Step 2: Define the GCN model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Step 3: Prepare the data for training
x = torch.tensor(data, dtype=torch.float)
y_distance = torch.tensor(df['FusionDistance'].astype(float).values, dtype=torch.float)  # Assuming the distance parameter is in 'Distance' column
y_mass = torch.tensor(df['FusionFSR'].astype(float).values, dtype=torch.float)  # Assuming the mass parameter is in 'Mass' column
num_nodes = x.shape[0]  # Get the number of nodes

data = Data(x=x, edge_index=edge_index, y_distance=y_distance, y_mass=y_mass, num_nodes=num_nodes)
loader = DataLoader([data], batch_size=1)


# Step 4: Train the model
train_losses = []  # List to store the training losses
train_accuracies = []  # List to store the training accuracies
input_dim = data.num_features
hidden_dim = 64  # Adjust the hidden dimension as needed
output_dim = 1  # Distance and mass are single-value predictions

model = GCN(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

model.train()
for epoch in range(150):
    for batch in loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y_distance) + criterion(out, batch.y_mass)  # Combined loss for distance and mass
        loss.backward()
        optimizer.step()

    # Calculate accuracy and loss
    acc = 0.0  # Calculate the accuracy for the current epoch
    loss = 0.0  # Calculate the loss for the current epoch

    # Append accuracy and loss to the respective lists
    train_accuracies.append(acc)
    train_losses.append(loss)

# Step 5: Save the trained model
torch.save(model.state_dict(), 'gcn_model.pt')

# Step 6: Download the model
from google.colab import files
files.download('gcn_model.pt')


**NEW**


In [ ]:
#import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
!pip install torch_geometric
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
import gspread
from oauth2client.service_account import ServiceAccountCredentials
#mount the google drive
from google.colab import drive
drive.mount('/content/gdrive')
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
creds, _ = default()
gc = gspread.authorize(creds)

# Step 1: Read data from Google Spreadsheet
spreadsheet_name = 'FYP (RoboLab)'  # Replace with your spreadsheet name
worksheet_name = 'Sheet1'  # Replace with your worksheet name

# Open the specified spreadsheet and worksheet
spreadsheet = gc.open(spreadsheet_name)
worksheet = spreadsheet.worksheet(worksheet_name)

# Extract the data from the worksheet
data = worksheet.get_all_values()
headers = data[0]
data = data[1:]

# Convert data to a DataFrame
df = pd.DataFrame(data, columns=headers)
data = df.iloc[:, 2:].values.astype(float)  # Extract columns 3 to 11 (sensor readings)
# edge_index = torch.tensor([[0, 1, 2, 3],
#                            [1, 0, 3, 2],
#                            [4, 5, 6],
#                            [5, 4, 6],
#                            [7, 8, 9],
#                            [8, 7, 9]])
edge_index = torch.tensor([
    [0, 2], [0, 3], [0, 4], [0, 5],  # Sensor fusion 1 (0) connected to sensors 2, 3, 4, 5
    [1, 2], [1, 3], [1, 4], [1, 5],  # Sensor fusion 2 (1) connected to sensors 2, 3, 4, 5
    [6, 8], [6, 9], [6, 10],         # Sensor fusion 3 (6) connected to sensors 8, 9, 10
    [7, 8], [7, 9], [7, 10],         # Sensor fusion 4 (7) connected to sensors 8, 9, 10
])

edge_index = edge_index.t().contiguous()  # Transpose and make contiguous



# Step 2: Define the GCN model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Step 3: Prepare the data for training
x = torch.tensor(data, dtype=torch.float)
y_distance = torch.tensor(df['FusionDistance'].astype(float).values, dtype=torch.float)  # Assuming the distance parameter is in 'Distance' column
y_mass = torch.tensor(df['FusionFSR'].astype(float).values, dtype=torch.float)  # Assuming the mass parameter is in 'Mass' column
num_nodes = x.shape[0]  # Get the number of nodes

data = Data(x=x, edge_index=edge_index, y_distance=y_distance, y_mass=y_mass, num_nodes=num_nodes)
loader = DataLoader([data], batch_size=1)


# Step 4: Train the model
train_losses = []  # List to store the training losses
train_accuracies = []  # List to store the training accuracies
input_dim = data.num_features
hidden_dim = 64  # Adjust the hidden dimension as needed
output_dim = 1  # Distance and mass are single-value predictions

model = GCN(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

model.train()
for epoch in range(200):
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    num_samples = 0

    def compute_accuracy(pred, target_distance, target_mass):
      pred = pred.round()
      correct_distance = pred.eq(target_distance.view_as(pred)).sum().item()
      correct_mass = pred.eq(target_mass.view_as(pred)).sum().item()
      total = target_distance.numel() + target_mass.numel()
      accuracy_distance = correct_distance / target_distance.numel()
      accuracy_mass = correct_mass / target_mass.numel()
      accuracy = (accuracy_distance + accuracy_mass) / 2  # Average accuracy
      return accuracy

    for batch in loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y_distance) + criterion(out, batch.y_mass)  # Combined loss for distance and mass
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * batch.num_graphs
        epoch_accuracy += compute_accuracy(out, batch.y_distance, batch.y_mass) * batch.num_graphs
        num_samples += batch.num_graphs

    epoch_loss /= num_samples
    epoch_accuracy /= num_samples

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

# Step 5: Save the trained model
torch.save(model.state_dict(), 'GCNmodel.pt')

# Step 6: Download the model
from google.colab import files
files.download('GCNmodel.pt')
print('Downloaded')


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([239])) that is different to the input size (torch.Size([239, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded
